# Welcome to RAG week!! (OpenRouter Version)

## Expert Knowledge Worker

### A question answering Assistant that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The AI assistant needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

This first implementation will use a simplistic, brute-force type of RAG..

**Modified to work with OpenRouter API**

In [1]:
import os
import glob
from dotenv import load_dotenv
from pathlib import Path
import gradio as gr
from openai import OpenAI

In [14]:
# Setting up OpenRouter
load_dotenv(override=True)

# OpenRouter configuration
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:8]}")
else:
    print("OpenRouter API Key not set")

MODEL = "deepseek/deepseek-chat"

# Initialize OpenAI client with OpenRouter endpoint
openai = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key,
)

OpenRouter API Key exists and begins sk-or-v1


In [15]:
print(f"Using model: {MODEL}")

Using model: deepseek/deepseek-chat


### Let's read in all employee data into a dictionary

In [4]:
knowledge = {}

filenames = glob.glob("knowledge-base/employees/*")

for filename in filenames:
    name = Path(filename).stem.split(' ')[-1]
    print(name, filename)
    with open(filename, "r", encoding="utf-8") as f:
        knowledge[name.lower()] = f.read()

Kim knowledge-base/employees/David Kim.md
Patel knowledge-base/employees/Nina Patel.md
Sharma knowledge-base/employees/Priya Sharma.md
Chen knowledge-base/employees/Alex Chen.md
Chen knowledge-base/employees/Robert Chen.md
Spencer knowledge-base/employees/Oliver Spencer.md
Foster knowledge-base/employees/Amanda Foster.md
Tran knowledge-base/employees/Emily Tran.md
Blake knowledge-base/employees/Jordan Blake.md
Zhang knowledge-base/employees/Kevin Zhang.md
Thompson knowledge-base/employees/Maya Thompson.md
Adams knowledge-base/employees/Jennifer Adams.md
Liu knowledge-base/employees/Jessica Liu.md
Lancaster knowledge-base/employees/Avery Lancaster.md
Thompson knowledge-base/employees/Maxine Thompson.md
Park knowledge-base/employees/Daniel Park.md
Greene knowledge-base/employees/Samantha Greene.md
Johnson knowledge-base/employees/Marcus Johnson.md
Thomson knowledge-base/employees/Alex Thomson.md
O'Brien knowledge-base/employees/Michael O'Brien.md
Martinez knowledge-base/employees/Rachel 

In [5]:
# Show available employees
print(f"Loaded {len(knowledge)} employee records")
print("Available employees:", list(knowledge.keys()))

Loaded 30 employee records
Available employees: ['kim', 'patel', 'sharma', 'chen', 'spencer', 'foster', 'tran', 'blake', 'zhang', 'thompson', 'adams', 'liu', 'lancaster', 'park', 'greene', 'johnson', 'thomson', "o'brien", 'martinez', 'williams', 'rivera', 'trenton', 'anderson', 'harper', 'rodriguez', 'wilson', 'bishop', 'carter', 'brooks', 'walker']


In [6]:
# Load product data
filenames = glob.glob("knowledge-base/products/*")

for filename in filenames:
    name = Path(filename).stem
    print(name, filename)
    with open(filename, "r", encoding="utf-8") as f:
        knowledge[name.lower()] = f.read()

Rellm knowledge-base/products/Rellm.md
Claimllm knowledge-base/products/Claimllm.md
Bizllm knowledge-base/products/Bizllm.md
Lifellm knowledge-base/products/Lifellm.md
Healthllm knowledge-base/products/Healthllm.md
Markellm knowledge-base/products/Markellm.md
Homellm knowledge-base/products/Homellm.md
Carllm knowledge-base/products/Carllm.md


In [7]:
print(f"Total knowledge base entries: {len(knowledge)}")
print("All available keys:", list(knowledge.keys()))

Total knowledge base entries: 38
All available keys: ['kim', 'patel', 'sharma', 'chen', 'spencer', 'foster', 'tran', 'blake', 'zhang', 'thompson', 'adams', 'liu', 'lancaster', 'park', 'greene', 'johnson', 'thomson', "o'brien", 'martinez', 'williams', 'rivera', 'trenton', 'anderson', 'harper', 'rodriguez', 'wilson', 'bishop', 'carter', 'brooks', 'walker', 'rellm', 'claimllm', 'bizllm', 'lifellm', 'healthllm', 'markellm', 'homellm', 'carllm']


In [8]:
SYSTEM_PREFIX = """
You represent Insurellm, the Insurance Tech company.
You are an expert in answering questions about Insurellm; its employees and its products.
You are provided with additional context that might be relevant to the user's question.
Give brief, accurate answers. If you don't know the answer, say so.

Relevant context:
"""

In [9]:
def get_relevant_context(message):
    text = ''.join(ch for ch in message if ch.isalpha() or ch.isspace())
    words = text.lower().split()
    return [knowledge[word] for word in words if word in knowledge]

In [10]:
def additional_context(message):
    relevant_context = get_relevant_context(message)
    if not relevant_context:
        result = "There is no additional context relevant to the user's question."
    else:
        result = "The following additional context might be relevant in answering the user's question:\n\n"
        result += "\n\n".join(relevant_context)
    return result

In [11]:
def chat(message, history):
    system_message = SYSTEM_PREFIX + additional_context(message)
    print(system_message)
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    print(messages)

    try:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=1000,
            temperature=0
        )
        print(response)
        return response.choices[0].message.content
    except Exception as e:
        print(e)
        return f"Error: {str(e)}"

## Test the chat function

In [12]:
# Test the chat function
test_response = chat("Who is Lancaster?", [])
print(test_response)


You represent Insurellm, the Insurance Tech company.
You are an expert in answering questions about Insurellm; its employees and its products.
You are provided with additional context that might be relevant to the user's question.
Give brief, accurate answers. If you don't know the answer, say so.

Relevant context:
The following additional context might be relevant in answering the user's question:

# Avery Lancaster

## Summary
- **Date of Birth**: March 15, 1985
- **Job Title**: Co-Founder & Chief Executive Officer (CEO)
- **Location**: San Francisco, California
- **Current Salary**: $225,000  

## Insurellm Career Progression
- **2015 - Present**: Co-Founder & CEO  
  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  

- **2013 - 2015**:

## Launch Gradio Interface

In [13]:
# Launch Gradio chat interface
view = gr.ChatInterface(
    chat,
    type="messages",
    title="Insurellm Knowledge Assistant (OpenRouter)",
    description=f"Ask questions about Insurellm employees and products. Using {MODEL} via OpenRouter."
).launch(inbrowser=True, share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.



You represent Insurellm, the Insurance Tech company.
You are an expert in answering questions about Insurellm; its employees and its products.
You are provided with additional context that might be relevant to the user's question.
Give brief, accurate answers. If you don't know the answer, say so.

Relevant context:
There is no additional context relevant to the user's question.
[{'role': 'system', 'content': "\nYou represent Insurellm, the Insurance Tech company.\nYou are an expert in answering questions about Insurellm; its employees and its products.\nYou are provided with additional context that might be relevant to the user's question.\nGive brief, accurate answers. If you don't know the answer, say so.\n\nRelevant context:\nThere is no additional context relevant to the user's question."}, {'role': 'user', 'content': 'who is alex?'}]
ChatCompletion(id='gen-1768069969-lFZKyVqXWSD0Xi8KIDg7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessag